## Ensuring Reproducibility

In [ ]:
import torch
import random
import numpy as np

# Set a fixed seed value for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Loading dataset

In [ ]:
# import opendatasets as od

# kaggle_data_url = "https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data"
# save_to = "datasets/jigsaw-toxic-comment"
# od.download(kaggle_data_url, save_to)

In [ ]:
import pandas as pd

data = pd.read_csv("datasets/jigsaw-toxic-comment/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
data = pd.DataFrame(
    {
        "text": data.iloc[:, 1].values.tolist(),
        "labels": data.iloc[:, 2:].values.tolist(),
    }
)

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# Load the Jigsaw Toxic Comment Classification dataset
dataset = Dataset.from_pandas(data)
train_test = dataset.train_test_split(test_size=0.3)
eval_test = train_test["test"].train_test_split(test_size=0.5)

jigsaw = DatasetDict(
    {
        "train": train_test["train"],
        "eval": eval_test["train"],
        "test": eval_test["test"],
    }
)
jigsaw

## Loading pretrained model

In [ ]:
# Load the pre-trained transformer model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

In [ ]:
# Freeze the pre-trained model parameters
for param in transformer_model.parameters():
    param.requires_grad = False

## Preparing data

In [ ]:
# Set up the data collator and dataloaders
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

max_length = 512
batch_size = 64

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=max_length)


# train_dataset = (jigsaw["train"].shuffle(seed=SEED).select([i for i in list(range(3000))]))
# eval_dataset = jigsaw["eval"].shuffle(seed=SEED).select([i for i in list(range(300))])
# test_dataset = jigsaw["test"].select([i for i in list(range(300, 600))])

train_dataset = jigsaw["train"].shuffle(seed=SEED)
eval_dataset = jigsaw["test"].shuffle(seed=SEED)
test_dataset = jigsaw["test"]

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_eval = eval_dataset.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_test = test_dataset.map(preprocess_function, batched=True, remove_columns=["text"])

train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=batch_size, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_eval, batch_size=batch_size, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_test, batch_size=batch_size, collate_fn=data_collator)

In [ ]:
# Move the model to the GPU (if available)
# device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Designing classification head

In [ ]:
# Define the custom classification head
class ClassificationHead(nn.Module):
    def __init__(self, transformer_model, num_classes):
        super().__init__()
        self.transformer_model = transformer_model.to(device)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(transformer_model.config.hidden_size, num_classes).to(device)

    def forward(self, input_ids, attention_mask):
        output = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)[0]
        output = self.dropout(output[:, 0])  # Take the CLS token representation
        output = self.classifier(output)
        return output

    # def __init__(self, transformer_model, num_classes):
    #     super().__init__()
    #     self.transformer_model = transformer_model.to(device)
    #     self.pre_classifier = torch.nn.Linear(768, 768)
    #     self.dropout = torch.nn.Dropout(0.1)
    #     self.classifier = torch.nn.Linear(768, 6)

    # def forward(self, input_ids, attention_mask, token_type_ids):
    #     output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
    #     hidden_state = output_1[0]
    #     pooler = hidden_state[:, 0]
    #     pooler = self.pre_classifier(pooler)
    #     pooler = torch.nn.Tanh()(pooler)
    #     pooler = self.dropout(pooler)
    #     output = self.classifier(pooler)
    #     return output

## Tuning model

In [ ]:
# Set hyperparameters
num_classes = 6
learning_rate = 2e-5
num_epochs = 5

In [ ]:
# Create the classification model
model = ClassificationHead(transformer_model, num_classes)

# Set up the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets.float())

In [ ]:
from pycm import MultiLabelCM
from sklearn import metrics


def train_epoch(model, train_dataloader, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        output = model(input_ids, attention_mask)
        loss = loss_fn(output, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss / len(train_dataloader)


def eval_epoch(model, eval_dataloader, loss_fn, device):
    model.eval()
    eval_loss = 0
    y_preds = []
    for batch in eval_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        output = model(input_ids, attention_mask)
        loss = loss_fn(output, labels)
        eval_loss += loss.item()
        y_preds.extend(
            [list(map(int, l)) for l in (output.sigmoid().round()).detach().tolist()]
        )
    return eval_loss / len(eval_dataloader), y_preds


def test_model(model, test_dataloader, loss_fn, device):
    model.eval()
    test_loss = 0
    y_preds = []
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        output = model(input_ids, attention_mask)
        loss = loss_fn(output, labels)
        test_loss += loss.item()
        y_preds.extend(
            [list(map(int, l)) for l in (output.sigmoid().round()).detach().tolist()]
        )
    test_loss /= len(test_dataloader)
    test_cm = MultiLabelCM(
        actual_vector=[
            {i for i, x in enumerate(sublist) if x == 1}
            for sublist in test_dataloader.dataset["labels"]
        ],
        predict_vector=[
            {i for i, x in enumerate(sublist) if x == 1} for sublist in y_preds
        ],
    )
    test_overall_acc = metrics.accuracy_score(
        test_dataloader.dataset["labels"], y_preds
    )
    return test_loss, test_overall_acc, test_cm, y_preds


def train_model(
    model,
    train_dataloader,
    eval_dataloader,
    test_dataloader,
    optimizer,
    loss_fn,
    device,
    num_epochs,
):
    stats = {}
    for epoch in range(num_epochs):
        tloss = train_epoch(model, train_dataloader, optimizer, loss_fn, device)
        eloss, y_preds = eval_epoch(model, eval_dataloader, loss_fn, device)
        eval_overall_acc = metrics.accuracy_score(eval_dataloader.dataset["labels"], y_preds)
        evaluation_cm = MultiLabelCM(
            actual_vector=[
                {i for i, x in enumerate(l) if x == 1}
                for l in eval_dataloader.dataset["labels"]
            ],
            predict_vector=[
                {i for i, x in enumerate(l) if x == 1} for l in y_preds
            ],
        )

        stats[f"epoch_{epoch+1}"] = {
            "training_loss": tloss,
            "validation_loss": eloss,
            f"epoch{epoch+1}_pred": y_preds,
            "validation_metrics": evaluation_cm,
        }

        print(
            f"Epoch = {epoch+1}/{num_epochs}\t Training Loss = {tloss:.2f}\t Validation Loss = {eloss:.2f}\t Validation Accuracy = {eval_overall_acc:.2f}"
        )

    test_loss, test_overall_acc, test_cm, test_preds = test_model(
        model, test_dataloader, loss_fn, device
    )
    stats["test_loss"] = test_loss
    stats["test_metrics"] = test_cm
    stats["test_pred"] = test_preds

    stats["eval_labels"] = eval_dataloader.dataset["labels"]
    stats["test_labels"] = test_dataloader.dataset["labels"]

    print(f"\nTest Accuracy = {test_overall_acc:.2f}")

    return stats

In [ ]:
# Train the model and evaluate on test set
stats = train_model(
    model,
    train_dataloader,
    eval_dataloader,
    test_dataloader,
    optimizer,
    loss_fn,
    device,
    num_epochs
)

## Saving the results

In [ ]:
import pickle

with open(f"results/jigsaw_bert.pickle", "wb") as file:
    pickle.dump(stats, file)

# # with open("jigsaw_bert.pickle", "rb") as file:
# #     stats = pickle.load(file)

## Model Parameters

In [ ]:
from torch.nn.utils import parameters_to_vector
from prettytable import PrettyTable


def count_parameters_per_layer(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(
        f"Total Number of Parameters: {parameters_to_vector(model.parameters()).numel()}"
    )
    print(f"Total Trainable Params: {total_params}")


count_parameters_per_layer(model)